In [10]:
import pandas as pd
import datetime as dt


PATH = '/path/to/your/data/'

mydateparser = lambda x: dt.datetime.strptime(x, "%Y-%m-%d %H:%M:%S")

account_id_type = dict()

account_id_type = {'account_id': str,
                   'post_id': str}

fb_type = {'account_id': str,
           'account_username': str,
           'fan': str,
           'post_id': str}

#Facebook
df_fb1 = pd.read_csv(PATH+"Facebook_Data_Q1.csv", dtype=fb_type, parse_dates=['created_at'], date_parser=mydateparser)
df_fb2 = pd.read_csv(PATH+"Facebook_Data_Q2.csv", dtype=fb_type, parse_dates=['created_at'], date_parser=mydateparser)
df_fb3 = pd.read_csv(PATH+"Facebook_Data_Q3.csv", dtype=fb_type, parse_dates=['created_at'], date_parser=mydateparser)
df_fb4 = pd.read_csv(PATH+"Facebook_Data_Q4.csv", dtype=fb_type, parse_dates=['created_at'], date_parser=mydateparser)
df_fb = df_fb1.append(df_fb2, ignore_index = True)
df_fb = df_fb.append(df_fb3, ignore_index = True)
df_fb = df_fb.append(df_fb4, ignore_index = True)

#IG, TW, YT
df_ig = pd.read_csv(PATH+"Instagram_Data.csv", dtype=account_id_type, parse_dates=['created_at'], date_parser=mydateparser)
df_tw = pd.read_csv(PATH+"Twitter_Data.csv", dtype=account_id_type, parse_dates=['created_at'], date_parser=mydateparser)
df_yt = pd.read_csv(PATH+"YouTube_Data.csv", dtype=account_id_type, parse_dates=['created_at'], date_parser=mydateparser)


# fb_cat_fanrange_funnel
df_cat_fanrange = pd.read_csv(PATH+'FB_cat_fanrange_funnel.csv', encoding='utf-8')




In [11]:
# calculate new engagement each channel
df_fb['new_engagement'] = df_fb['engagement']
df_ig['new_engagement'] = df_ig[['like', 'comment','view','share_on_facebook']].apply(sum, axis=1)
df_tw['new_engagement'] = df_tw[['favorite', 'retweet','share_on_facebook','reply']].apply(sum, axis=1)
df_yt['new_engagement'] = df_yt[['like', 'dislike','favorite','comment','view','share_on_facebook']].apply(sum, axis=1)

# combine 14 column of 4 channel to 1 dataframe
selected_column = ['post_id', 'channel', 'account_id', \
    'account_username', 'account_display_name', 'link', \
    'created_at', 'message', 'new_engagement', 'positive', 'neutral', 'negative', 'fan']

df_all = df_fb[selected_column].append(df_ig[selected_column], ignore_index = True)
df_all = df_all.append(df_tw[selected_column], ignore_index = True)
df_all = df_all.append(df_yt[selected_column], ignore_index = True)


# Drop row that has '\N'
df_all = df_all[df_all['fan'] != '\\N']

# change 'fan' column to int
df_all['fan'] = df_all['fan'].astype('int64')
df_all.info(verbose=True, show_counts=True)


# create dataframe that contain account_id, account_display_name
df_acc = df_all[['account_id','account_display_name']]
df_acc = df_acc.drop_duplicates(subset='account_id', keep='last')

# threshold for time range
bins = list(range(0,25))

time_range = ["00:00", "01:00", "02:00", "03:00", "04:00", "05:00", "06:00", "07:00", "08:00", "09:00",
              "10:00", "11:00", "12:00", "13:00", "14:00", "15:00", "16:00", "17:00", "18:00", "19:00", 
              "20:00", "21:00", "22:00", "23:00"]

# create new column to tell created time of each post?
df_all['post_time_range'] = pd.cut(df_all.created_at.dt.hour,       
                             bins, 
                             labels=time_range,  
                             right=False)

# create day column
df_all['day'] = df_all['created_at'].dt.day_name()

# Define column 'day' as categories type which order the seq of day name
cats = [ 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
from pandas.api.types import CategoricalDtype
cat_type = CategoricalDtype(categories=cats, ordered=True)
df_all['day'] = df_all['day'].astype(cat_type)





<class 'pandas.core.frame.DataFrame'>
Int64Index: 4087162 entries, 0 to 4087238
Data columns (total 13 columns):
 #   Column                Non-Null Count    Dtype         
---  ------                --------------    -----         
 0   post_id               4087162 non-null  object        
 1   channel               4087162 non-null  object        
 2   account_id            4087162 non-null  object        
 3   account_username      832211 non-null   object        
 4   account_display_name  4080641 non-null  object        
 5   link                  4087047 non-null  object        
 6   created_at            4087162 non-null  datetime64[ns]
 7   message               2647164 non-null  object        
 8   new_engagement        4087162 non-null  int64         
 9   positive              4087162 non-null  int64         
 10  neutral               4087162 non-null  int64         
 11  negative              4087162 non-null  int64         
 12  fan                   4087162 non-null  in

In [12]:
# export file for mapping account with file Account_Mapping_NoCat.csv by using vlookup in excel  
df_all_account = df_all[['channel', 'account_id', 'account_username', 'account_display_name']]
df_all_account.drop_duplicates(inplace=True)
df_all_account.to_csv('df_all_account.csv', index=False, encoding='utf-8')

/var/folders/zr/_h829tpn5zq19y29l_cbntzm0000gn/T/ipykernel_69379/3173307639.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all_account.drop_duplicates(inplace=True)


In [13]:
#Animated scatter plotter for avg view and fan
df_view_vs_fan = df_yt[['account_display_name', 'view', 'created_at', 'fan']]

# create month column
df_view_vs_fan['month'] = df_view_vs_fan['created_at'].dt.month_name()

month_cats = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
from pandas.api.types import CategoricalDtype
cat_type = CategoricalDtype(categories=month_cats, ordered=True)
df_view_vs_fan['month'] = df_view_vs_fan['month'].astype(cat_type)


# Get avg_view
df_view_vs_fan = df_view_vs_fan.groupby(['account_display_name', 'month'])['view', 'fan'].agg({'view':'sum', 'fan': 'count'}).reset_index()
df_view_vs_fan = df_view_vs_fan.rename(columns={'fan':'count'})
df_view_vs_fan['avg_view'] = df_view_vs_fan['view'] // df_view_vs_fan['count']

df_yt_fan = df_yt[['account_display_name', 'created_at', 'fan']]

# create month column
df_yt_fan['month'] = df_yt_fan['created_at'].dt.month_name()

month_cats = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
from pandas.api.types import CategoricalDtype
cat_type = CategoricalDtype(categories=month_cats, ordered=True)
df_yt_fan['month'] = df_yt_fan['month'].astype(cat_type)

# Get latest fan
df_yt_fan = df_yt_fan.sort_values(by=['account_display_name', 'month', 'created_at']).groupby(['account_display_name', 'month']).tail(1)


# Merge dataframe
df_view_vs_fan = df_view_vs_fan.merge(right=df_yt_fan, left_on=['account_display_name', 'month'], right_on=['account_display_name', 'month'], how='inner')

# create column type_fan
def classify_fan_amount(row):

    if row['fan'] >= 100000 and row['fan'] <= 999999:
        type_fan = 'Silver'
    elif row['fan'] > 999999 and row['fan'] <= 10000000:
        type_fan = 'Gold'
    elif row['fan'] > 10000000:
        type_fan = 'Diamond'
    else:
        type_fan = 'NA'

    return type_fan

df_view_vs_fan['type_fan'] = df_view_vs_fan.apply(classify_fan_amount, axis=1)
df_view_vs_fan = df_view_vs_fan.loc[df_view_vs_fan['type_fan'] != 'NA']

fan_cats = ['Diamond', 'Gold', 'Silver', 'Filtered by dropdown']
fan_type = CategoricalDtype(categories=fan_cats, ordered=True)
df_view_vs_fan['type_fan'] = df_view_vs_fan['type_fan'].astype(fan_type)

df_view_vs_fan = df_view_vs_fan.sort_values(by=['type_fan', 'account_display_name', 'month'])

# add missing month name each account id by using groupby categorical column(month)
df_view_vs_fan = df_view_vs_fan.groupby(['account_display_name', 'month']).agg(lambda x:x).reset_index()
df_view_vs_fan.loc[:,['view', 'count', 'avg_view', 'created_at', 'fan']] = df_view_vs_fan.loc[:,['view', 'count', 'avg_view', 'created_at', 'fan']].fillna(0)
df_view_vs_fan.loc[:,['type_fan']] = df_view_vs_fan.loc[:,['type_fan']].fillna('Silver')

df_view_vs_fan = df_view_vs_fan.sort_values(by=['type_fan', 'account_display_name', 'month'])

# Export to csv file
df_view_vs_fan.to_csv('df_view_vs_fan.csv', encoding='utf-8')


/var/folders/zr/_h829tpn5zq19y29l_cbntzm0000gn/T/ipykernel_69379/48749063.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_view_vs_fan['month'] = df_view_vs_fan['created_at'].dt.month_name()
/var/folders/zr/_h829tpn5zq19y29l_cbntzm0000gn/T/ipykernel_69379/48749063.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_view_vs_fan['month'] = df_view_vs_fan['month'].astype(cat_type)
/var/folders/zr/_h829tpn5zq19y29l_cbntzm0000gn/T/ipykernel_69379/48749063.py:14: FutureWarning: Indexing with multiple ke

In [15]:
# line chart fan in each month of each account ID
df_map_all_platform = pd.read_csv(PATH+'Account_mapping_v2.csv', encoding='utf-8')


df_fan_each_month = df_all[['channel', 'account_id', 'created_at', 'fan']].merge(right=df_map_all_platform[['account_id', 'mapped_name']], 
                                                                                right_on='account_id',
                                                                                left_on='account_id',
                                                                                how='inner')[['mapped_name', 'channel', 'created_at', 'fan']]

# create month column
df_fan_each_month['month'] = df_fan_each_month['created_at'].dt.month_name()

df_fan_each_month = df_fan_each_month[['mapped_name', 'channel', 'month', 'created_at', 'fan']]

month_cats = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
from pandas.api.types import CategoricalDtype
cat_type = CategoricalDtype(categories=month_cats, ordered=True)
df_fan_each_month['month'] = df_fan_each_month['month'].astype(cat_type)


# get lastest fan from last date of each month
df_fan_each_month = df_fan_each_month.sort_values(by=['mapped_name', 'channel', 'month', 'created_at'], ascending=True).groupby(['mapped_name', 'channel', 'month']).tail(1)

# add missing month name each account id by using groupby categorical column(month)
df_fan_each_month = df_fan_each_month.groupby(['mapped_name', 'channel', 'month'])['fan'].agg(lambda x:x).reset_index()

import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.graph_objs import Layout

# data for drop down list to filter mapped_name
df_unique_name_channel= df_fan_each_month[['mapped_name', 'channel']].drop_duplicates()

# export to csv
df_unique_name_channel.to_csv('df_unique_name_channel.csv', encoding='utf-8')
df_fan_each_month.to_csv('df_fan_each_month.csv', encoding='utf-8')


In [17]:
# Bar chart Fan+Engagement
df_engagement_fan = df_all.loc[:, ['channel', 'account_id', 'created_at', 'new_engagement', 'fan']]

selected_column = ['channel', 'account_id', 'new_engagement', 'fan']

# sum new_engagement & count post each account
df_avg_engagement = df_engagement_fan[selected_column].groupby(['channel', 'account_id']).agg({'new_engagement': 'sum', 'fan': 'count'}).reset_index()
df_avg_engagement = df_avg_engagement.rename(columns={'fan':'post_count'})

# create average engagement per post 
df_avg_engagement['value'] = df_avg_engagement['new_engagement'] / df_avg_engagement['post_count']
df_avg_engagement['type_value'] = 'average_engagement_per_post'


# get the latest fan amount each account
df_fan_by_account = df_engagement_fan.sort_values(by=['channel', 'account_id', 'created_at'], ascending=False).drop_duplicates(subset=['channel', 'account_id'])
df_fan_by_account['type_value'] = 'fan'
df_fan_by_account = df_fan_by_account.rename(columns={'fan':'value'})

# find 'total' = average engagement + fan in order to sort data
df_total = df_avg_engagement.merge(df_fan_by_account[['account_id', 'value']], left_on='account_id', right_on='account_id')
df_total['total'] = df_total['value_x'] + df_total['value_y']

# combine df_avg_engagement + df_fan_by_account
df_channel_account_top_engagement = df_avg_engagement[['channel', 'account_id', 'type_value', 'value']].append(df_fan_by_account[['channel', 'account_id','type_value','value']], ignore_index=True )

# add 'total' column to df_channel_account_top_engagement
df_channel_account_top_engagement = df_channel_account_top_engagement.merge(df_total[['account_id', 'total']], left_on='account_id', right_on='account_id', how='left')

# add 'account_display_name' column to df_channel_account_top_engagement
df_channel_account_top_engagement = df_channel_account_top_engagement.merge(df_acc, left_on='account_id', right_on='account_id', how='left')

# sort value
df_channel_account_top_engagement = df_channel_account_top_engagement.sort_values(by=['channel', 'total', 'type_value'], ascending=False)
df_channel_account_top_engagement

# Export to csv
df_channel_account_top_engagement.to_csv('df_channel_account_top_engagement.csv', index=False ,encoding='utf-8')

In [18]:
# Heatmap chart day vs time
df_channel_posttime_engagement = df_all.loc[:, ['channel', 'day', 'post_time_range', 'new_engagement']]

df_channel_posttime_engagement['post_time_range'] = df_channel_posttime_engagement['post_time_range'].astype(str)
df_channel_posttime_engagement = df_channel_posttime_engagement.groupby(['channel', 'day', 'post_time_range']).agg('sum').reset_index()

# create pivot table to be input of heatmap
df_day_vs_time = pd.pivot_table(df_channel_posttime_engagement, index=['channel', 'day'], columns='post_time_range', values='new_engagement')

# reset index before export to csv
df_day_vs_time = df_day_vs_time.rename_axis(None, axis=1).reset_index()

df_day_vs_time.to_csv('df_day_vs_time.csv', index=False ,encoding='utf-8')

In [19]:
# Donut chart popular by engagement and influencer
# create pivot table to be input of donut chart
df_channel_engagement = df_all.loc[:, ['channel', 'account_id', 'new_engagement']]

df_channel_engagement = df_channel_engagement.groupby(['channel']).agg({'account_id': pd.Series.nunique, 'new_engagement': 'sum'}).sort_index(ascending=True).reset_index()

df_channel_engagement
df_channel_engagement.to_csv('df_channel_engagement.csv', index=False ,encoding='utf-8')

In [20]:
# Bar chart Video vs Image posting
df_ig_img_video = df_ig.loc[:, ['post_type', 'new_engagement', 'channel']]

df_ig_img_video = df_ig_img_video.groupby('post_type').agg({'new_engagement': 'sum', 'channel': 'count'}).reset_index()
df_ig_img_video = df_ig_img_video.rename(columns={'channel': 'post_count'})
df_ig_img_video['avg_engagement'] =  df_ig_img_video['new_engagement'] / df_ig_img_video['post_count']
df_ig_img_video = df_ig_img_video.sort_values(by='avg_engagement',ascending=False)

df_ig_img_video.to_csv('df_ig_img_video.csv', index=False, encoding='utf-8')


In [21]:
# Funnel chart for purchasing intention
df_fb_funnel = df_fb.loc[:, ['account_id', 'new_engagement', 'reaction', 'like', 'love', 'wow', 'haha', 'share', 'comment', 'tag_friend', 'purchase_intention']]

# sum by account_id
df_fb_funnel = df_fb_funnel.groupby('account_id').agg(sum).reset_index()

df_fb_funnel['total_engagement'] = df_fb_funnel['new_engagement'] + df_fb_funnel['tag_friend'] + df_fb_funnel['purchase_intention']

df_fb_funnel = df_fb_funnel[['account_id', 'total_engagement', 'reaction', 'share', 'comment', 'tag_friend', 'purchase_intention']]

# use pd.melt to change wide format to long format
df_fb_funnel = pd.melt(df_fb_funnel, id_vars='account_id', value_vars=['total_engagement', 'reaction', 'share', 'comment', 'tag_friend', 'purchase_intention'], var_name='type')



# merge with account_display_name
df_fb_funnel = df_fb_funnel.merge(df_acc, left_on='account_id', right_on='account_id', how='left')

# change 'type' column to be categorical type in order to sort value.
funnel_cats = ['total_engagement', 'reaction', 'share', 'comment', 'tag_friend', 'purchase_intention']
from pandas.api.types import CategoricalDtype
funnel_type = CategoricalDtype(categories=funnel_cats, ordered=True)
df_fb_funnel['type'] = df_fb_funnel['type'].astype(funnel_type)
df_fb_funnel = df_fb_funnel.sort_values(by=['account_id', 'type'])

# merge with category and fanrage
df_fb_funnel = df_fb_funnel.merge(right=df_cat_fanrange[['account_display_name', 'category', 'fan_range']], left_on='account_display_name', right_on='account_display_name', how='inner')


# export to csv
df_fb_funnel.to_csv('df_fb_funnel.csv', index=False, encoding='utf-8')

In [23]:
import pandas as pd
import re
import numpy as np

# function to print all the hashtags in a text
def extract_hashtags(text):
     
    # the regular expression
    regex = "#[(\w)|\u0E00-\u0E7F]+" 
     
    # extracting the hashtags
    hashtag_list = re.findall(regex, text)
    
    return hashtag_list   

def get_hashtags(rown):
    # Check is nan     
    if(rown['message'] != rown['message']):
        rown['message'] = ""
    return extract_hashtags(rown['message'])

def get_count_frequency(df,column):
    freq_dict = {}
    for rown in df[column]:
        for hashtags in rown:
            if (hashtags in freq_dict):
                freq_dict[hashtags] += 1
            else:
                freq_dict[hashtags] = 1
    return freq_dict

twitter_df = df_tw.reset_index()

twitter_df.info()
twitter_df.count()
twitter_df.shape

twitter_df['hashtags'] = np.empty((len(twitter_df), 0)).tolist()
twitter_df['hashtags'] = twitter_df.apply(get_hashtags,axis=1)
hashtags_dict = get_count_frequency(twitter_df,'hashtags')
hashtags_df = pd.DataFrame(
    {'hashtags': list(hashtags_dict.keys()),
     'total': list(hashtags_dict.values())
    })
hashtags_df[hashtags_df['total'] > 50].sort_values(by=['total'], ascending=False).head(5)
hashtags_df[hashtags_df['total'] > 50].sort_values(by=['total'], ascending=False).to_csv("twitter_hashtags.csv",index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253295 entries, 0 to 253294
Data columns (total 19 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   index                 253295 non-null  int64         
 1   post_id               253295 non-null  object        
 2   channel               253295 non-null  object        
 3   account_id            253295 non-null  object        
 4   account_username      253295 non-null  object        
 5   account_display_name  253295 non-null  object        
 6   link                  253295 non-null  object        
 7   created_at            253295 non-null  datetime64[ns]
 8   message               253295 non-null  object        
 9   favorite              253295 non-null  int64         
 10  retweet               253295 non-null  int64         
 11  share_on_facebook     253295 non-null  int64         
 12  engagement            253295 non-null  int64         
 13 